<a href="https://colab.research.google.com/github/gabipalacios/Python/blob/main/RiesgoZ2capa_V11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import io
from google.colab import files
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sn
from google.colab import drive
pd.options.display.float_format = '{:.2f}'.format

drive.mount('/content/drive')
origen = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_sinpagos1.csv')
marco = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_PreMarcoBCU.csv')

marco = marco.add_prefix('m_')
origen = origen.add_prefix('o_')

#Elimino campos que no tengo el dato de plataforma, pero si de Financial, son prestamos que si bien
#se originan en plataforma, se termienan prestamos por terceros. Tengo los datos de Monto, Cuota y tasa.
#cardo orig, seguro y cargo adm los saco porque además de faltar datos, son resultados que no aportan valor al modelo
origen = origen.drop(['o_monto', 'o_cuota', 'o_tasa', 'o_plazo', 'o_cargo_originacion', 'o_tiene_seguro', 'o_tiene_cargo_adm',
                      'o_clr_score_if', 'o_clr_bp_fecha', 'o_rpf_alert', 'o_cdr_tiene_quita', 'o_cdr_tiene_oca', 'o_cdr_tiene_oca.1',
                      'o_cdr_tiene_financiera', 'o_cdr_tiene_brou', 'o_cdr_tiene_banco_privado', 'o_cdr_sum_deudas_ui', 'o_cdr_periodo',
                      'o_cdr_peor_calificacion', 'o_cdr_max_contingencia_ui', 'o_Tipo_Producto', 'o_Tipo Basico', 'o_clr_fecha',
                      'o_Canal', 'o_no_of_desc', 'o_id_solicitud', 'o_DesembolsoLugar', 'o_estado_tc', 'o_estado_pa',
                      'o_estado_cliente', 'o_cargo', 'o_meses_con_pagos', 'o_dias_atraso_max', 'o_DesembolsoSubAge.',
                      'o_DesembolsoAgencia', 'o_DesembolsoAgencia.1', 'o_CuotasPagas', 'o_localidad', 'o_Atraso', 'o_Solicitud',
                      'o_Fecha', 'o_tiene_tel_fijo', 'o_tiene_recibo_sueldo', 'o_Id_Clliente'], axis=1)


df = pd.merge(origen, marco, left_on='o_Documento', right_on='m_Documento', how='inner')
df = df.set_index('m_Documento')

df = df.dropna(subset=['o_clr_score'])

# Use .loc for assignment to avoid SettingWithCopyWarning
df.loc[~df['o_dispositivo_marca'].isin(['Apple']), 'o_dispositivo_marca'] = 'Android'
df.loc[df['o_sist_op_familia'] != 'iOS', 'o_sist_op_familia'] = 'Android'
df.loc[df['o_dispositivo_marca'] != 'Apple', 'o_dispositivo_marca'] = 'Android'
df.loc[~df['o_navegador_familia'].isin(['Facebook', 'Instagram']), 'o_navegador_familia'] = 'Web'
df.loc[~df['o_departamento'].isin(['MONTEVIDEO', 'CANELONES']), 'o_departamento'] = 'RESTO'
df['o_Zero'] = np.where(df['o_Zero'] == 'Zero', 1, 0)
df['o_clr_accion'] = np.where(df['o_clr_accion'].isin(['C2', 'C1_3_3','C4','C1_2_3','C1_3_6','C1_3_1','C3']), df['o_clr_accion'], 'Varios')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Modificar el parámetro de la tabla df['hora'] donde se muestran las horas del día desde 0 a 23, son tipos de datos periódicos, utilizando el cálculo trigonométrico más adecuado para introducir el parámetro en un modelo predictivo.

# Convertir la columna 'hora' a valores numéricos entre 0 y 23
df['o_Hora'] = pd.to_numeric(df['o_Hora'])

# Calcular la representación sinusoidal de la hora
df['hora_sin'] = np.sin(2 * np.pi * df['o_Hora'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['o_Hora'] / 24)

# Eliminar la columna original 'hora'
df = df.drop('o_Hora', axis=1)


In [ ]:
df.columns

Index(['o_Documento', 'o_Día', 'o_Semana_año', 'o_Quincena', 'o_Num_Dia_Sem',
       'o_Zero', 'o_TEA', 'o_status_usuario', 'o_status', 'o_sist_op_familia',
       'o_relacion_laboral', 'o_plazo_hasta', 'o_nivel_educativo',
       'o_nivel_antiguedad_laboral', 'o_navegador_familia', 'o_monto_maximo',
       'o_mejor_esquema_pricing', 'o_ingreso_estimado', 'o_ingreso_declarado',
       'o_genero', 'o_edad', 'o_dispositivo_marca', 'o_departamento',
       'o_Cuotas', 'o_clr_score', 'o_clr_accion', 'o_cel_carrier', 'o_Capital',
       'm_Entidades', 'm_Exposición', 'm_Deuda', 'm_Rel_DeudaPrincipal',
       'm_Contingencia', 'm_Var_dueda_3M', 'm_Sin_Ficha', 'm_Malos',
       'm_Mejora', 'm_Grupo_b', 'm_Grupo_i', 'm_Q_irregular_12m', 'hora_sin',
       'hora_cos'],
      dtype='object')

In [ ]:
count_zero_monto_maximo = len(df[df['o_monto_maximo'] == 0])
print(f"Número de registros con monto_maximo igual a 0: {count_zero_monto_maximo}")
min_monto_maximo = df.loc[df['o_monto_maximo'] != 0, 'o_monto_maximo'].min()
df.loc[df['o_monto_maximo'] == 0, 'o_monto_maximo'] = min_monto_maximo
count_zero_monto_maximo = len(df[df['o_monto_maximo'] == 0])
print(f"Número de registros con monto_maximo igual a 0: {count_zero_monto_maximo}")

Número de registros con monto_maximo igual a 0: 60
Número de registros con monto_maximo igual a 0: 0


In [ ]:
df.describe()

,o_Documento,o_Día,o_Semana_año,o_Quincena,o_Num_Dia_Sem,o_Zero,o_TEA,o_plazo_hasta,o_nivel_antiguedad_laboral,o_monto_maximo,...,m_Rel_DeudaPrincipal,m_Contingencia,m_Var_dueda_3M,m_Sin_Ficha,m_Malos,m_Mejora,m_Q_irregular_12m,hora_sin,hora_cos,monto_maximo
count,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,...,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,0.00
mean,38282484.96,15.91,28.66,1.51,4.01,0.24,85.69,20.52,1.87,39056.37,...,0.59,0.64,0.75,0.27,0.15,0.28,1.06,-0.27,-0.26,NaN
std,11069716.15,8.13,19.44,0.50,1.73,0.42,17.15,2.26,1.08,23239.78,...,0.25,0.48,2.00,0.68,0.52,1.07,1.97,0.62,0.69,NaN
min,9531591.00,1.00,1.00,1.00,1.00,0.00,32.72,19.00,1.00,5200.00,...,0.00,0.00,-0.76,0.00,0.00,-5.00,0.00,-1.00,-1.00,NaN
25%,31482666.50,9.00,6.00,1.00,2.00,0.00,67.80,19.00,1.00,22475.00,...,0.41,0.00,-0.02,0.00,0.00,0.00,0.00,-0.87,-0.87,NaN
50%,40603097.00,16.00,38.00,2.00,4.00,0.00,89.30,19.00,1.00,35200.00,...,0.57,1.00,0.08,0.00,0.00,0.00,0.00,-0.50,-0.50,NaN
75%,47181084.75,23.00,46.00,2.00,5.00,0.00,98.96,24.00,3.00,50100.00,...,0.79,1.00,0.50,0.00,0.00,0.00,1.00,0.26,0.26,NaN
max,57905679.00,31.00,53.00,2.00,7.00,1.00,128.00,28.00,4.00,194900.00,...,1.00,1.00,10.00,3.00,3.00,6.00,12.00,1.00,1.00,NaN
